In [ ]:
import time
import copy
import logging
import networkx
from networkx import convert_matrix
from typing import NamedTuple, Optional, Tuple, List, Dict, TypeVar, TypedDict, cast

import numpy
import gym
from gym import spaces
from gym.utils import seeding

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from cyberbattle._env.shared_cyberbattle_env import EnvironmentBounds, AttackerGoal, DefenderGoal, DefenderConstraint
from cyberbattle._env.defender import DefenderAgent,ScanAndReimageCompromisedMachines
from cyberbattle.simulation.model import PortName, PrivilegeLevel
from cyberbattle.simulation import commandcontrol, model, actions
from cyberbattle._env.discriminatedunion import DiscriminatedUnion
from cyberbattle.agents.baseline import agent_wrapper as w
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.learner import Learner
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.agent_ddql as ddqla
import cyberbattle.agents.baseline.agent_dql as dqla
from cyberbattle.defender_agents.trainedDefender import *
import random

In [ ]:
random.seed(120394016)
%matplotlib inline
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
#ql
iteration_count = 10
training_episode_count = 10
eval_episode_count = 20
gamma_sweep = [
    0.015,  # about right
]

In [ ]:
import gym

# Create an instance of the CyberBattleChain environment without the defender
gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)


defender_name = "DefenderName" 

defender_agent = DefenderWrapper(gym_env,defender_name)

Pre-tuning defender

In [ ]:
from cyberbattle.agents.baseline.agent_wrapper import ActionTrackingStateAugmentation, StateAugmentation
from cyberbattle.agents.baseline.plotting import PlotTraining, plot_averaged_cummulative_rewards
import progressbar
import math
from cyberbattle.agents.baseline import learner
import sys
from cyberbattle.simulation.model import FirewallRule

ep = w.EnvironmentBounds.of_identifiers(
    maximum_total_credentials=22,
    maximum_node_count=22,
    identifiers=gym_env.identifiers
)
#cyberbattlechain_defender.render_as_fig()

In [ ]:
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
random.seed(120394016)
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=10
iteration_count=10
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)

In [ ]:
def run_training_with_hyperparameters(gym_env, environment_properties, statespace, actionspace,
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay):
    q_matrix = DefenderQMatrix(statespace, actionspace, gym_env)
    learner = DefenderQLearner(gym_env, defender_agent, environment_properties, q_matrix,
                               epsilon=epsilon, gamma=gamma, learning_rate=learning_rate)

    trained_defender = epsilon_greedy_defender_training(
        cyberbattle_gym_env=gym_env,
        environment_properties=environment_properties,
        learner=learner,
        episode_count=episode_count,  # Set the number of episodes for the training
        iteration_count=iteration_count,  # Set the number of iterations per episode
        epsilon=epsilon,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        epsilon_multdecay=epsilon_multdecay,
        verbosity=logging.INFO,
        render=True,
        plot_episodes_length=True,
        title="Defender Q-Learning Training"
    )
    total_reward = sum([sum(rewards) for rewards in trained_defender['all_episodes_rewards']])
    return total_reward

In [ ]:
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
random.seed(120394016)
# Parameter ranges
epsilon = .5
#gamma=np.linspace(0.05,0.09,10)
gamma= 0.0
#learning_rate=np.linspace(0.3,.35,10)
learning_rate=0.0
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=1000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0
episode_count=10
iteration_count=10
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.0
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])


total_reward=run_training_with_hyperparameters(gym_env, ep, statespace, actionspace,
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay)


In [ ]:
print(total_reward)

740, 620,660,460, 630, 540, 610, 970, 660, 690,840,540,440

Post-tuning defender

In [ ]:
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
random.seed(120394016)
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=10
iteration_count=10
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)
total_reward1=run_training_with_hyperparameters(gym_env, ep, statespace, actionspace,
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay)
total_reward1

810,840,550,800,690,440,760,670,590,690

In [ ]:
pip install scipy


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

pre_tuned_rewards = [620, 660, 460, 630, 540, 610, 660, 690, 540, 440]
post_tuned_rewards = [810, 840, 550, 800, 690, 440, 760, 670, 590, 690]

#Perform a Shapiro-Wilk test for normality
print("Normality Test (Shapiro-Wilk):")
print("Pre-Tuned:", stats.shapiro(pre_tuned_rewards))
print("Post-Tuned:", stats.shapiro(post_tuned_rewards))

#Perform Levene's test for equal variances
print("\nEqual Variance Test (Levene's):")
print(stats.levene(pre_tuned_rewards, post_tuned_rewards))

#Perform a t-test for the means of two independent samples
print("\nT-test:")
t_test_result = stats.ttest_ind(pre_tuned_rewards, post_tuned_rewards, equal_var=True)
print(t_test_result)

#If the assumptions of the t-test are not met, perform a Mann-Whitney U test
print("\nMann-Whitney U Test:")
mwu_test_result = stats.mannwhitneyu(pre_tuned_rewards, post_tuned_rewards)
print(mwu_test_result)

plt.figure(figsize=(10, 6))
plt.boxplot([pre_tuned_rewards, post_tuned_rewards], labels=['Pre-Tuned', 'Post-Tuned'])
plt.title('Comparison of Total Rewards Before and After Tuning')
plt.ylabel('Total Rewards')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(12, 7))

#Define colours for each boxplot and scatter
#box_colors = ['#1f77b4', '#ff7f0e']
box_colors = ['pink', 'lightblue']
scatter_colors = ['#3090C7', '#FFA500']

bp = ax.boxplot([pre_tuned_rewards, post_tuned_rewards], patch_artist=True,
                labels=['Pre-Tuned', 'Post-Tuned'],
                medianprops={'color': 'red', 'linewidth': 2},
                boxprops=dict(facecolor='lightgray', color='black', linewidth=2),
                whiskerprops={'color': 'black', 'linewidth': 1.5},
                capprops={'color': 'black', 'linewidth': 1.5},
                showfliers=False)  # Optionally hide the fliers

for patch, color in zip(bp['boxes'], box_colors):
    patch.set_facecolor(color)

# Scatter plot for individual data points
for i, data in enumerate([pre_tuned_rewards, post_tuned_rewards]):
    y = np.random.normal(1 + i, 0.04, size=len(data))  # Slight jitter for clarity
    ax.scatter(y, data, alpha=0.7, color=scatter_colors[i], edgecolor='black', zorder=2, s=50)

#Calculate and plot overall means for pre-tuned and post-tuned
overall_mean_pre = np.mean(pre_tuned_rewards)
overall_mean_post = np.mean(post_tuned_rewards)

ax.hlines(overall_mean_pre, xmin=0.8, xmax=1.2, colors='blue', linestyles='dashed', label='Mean Pre-Tuned')
ax.hlines(overall_mean_post, xmin=1.8, xmax=2.2, colors='green', linestyles='dashed', label='Mean Post-Tuned')

# Highlight maximum values with annotation
max_pre = max(pre_tuned_rewards)
max_post = max(post_tuned_rewards)

max_pre_index = pre_tuned_rewards.index(max_pre) + 1
max_post_index = post_tuned_rewards.index(max_post) + 1 + len(pre_tuned_rewards)  # Offset for post-tuned

ax.annotate('Max Pre-Tuned: {}'.format(max_pre),
            xy=(1, max_pre), xytext=(0.8, max_pre + 50),
            arrowprops=dict(facecolor='black', shrink=0.05),
            fontsize=12, color='black')

ax.annotate('Max Post-Tuned: {}'.format(max_post),
            xy=(2, max_post), xytext=(2.2, max_post - 50),
            arrowprops=dict(facecolor='black', shrink=0.05),
            fontsize=12, color='black')

ax.set_title('Comparison of Total Rewards Before and After Tuning', fontsize=16)
ax.set_ylabel('Total Rewards', fontsize=14)
ax.set_xticklabels(['Pre-Tuned', 'Post-Tuned'], fontsize=12)
ax.set_yticklabels(np.round(ax.get_yticks(), 2), fontsize=12)

ax.yaxis.grid(True, linestyle='--', which='major', color='grey', alpha=0.7)
ax.set_axisbelow(True)  # Ensure grid is below plot elements

plt.legend(loc='upper right', fontsize=12)
plt.tight_layout()

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(12, 7))

# Define colors for each boxplot and scatter
box_colors = ['pink', 'lightblue']
scatter_colors = ['#3090C7', '#FFA500']

bp = ax.boxplot([pre_tuned_rewards, post_tuned_rewards], patch_artist=True,
                labels=['Pre-Tuned', 'Post-Tuned'],
                medianprops={'color': 'red', 'linewidth': 2},
                boxprops=dict(facecolor='lightgray', color='black', linewidth=2),
                whiskerprops={'color': 'black', 'linewidth': 1.5},
                capprops={'color': 'black', 'linewidth': 1.5},
                showfliers=False)  # Optionally hide the fliers

# Coloring the boxes
for patch, color in zip(bp['boxes'], box_colors):
    patch.set_facecolor(color)

# Scatter plot for individual data points
for i, data in enumerate([pre_tuned_rewards, post_tuned_rewards]):
    y = np.random.normal(1 + i, 0.04, size=len(data))  # Slight jitter for clarity
    ax.scatter(y, data, alpha=0.7, color=scatter_colors[i], edgecolor='black', zorder=2, s=50)

# Calculate and plot overall means for pre-tuned and post-tuned
overall_mean_pre = np.mean(pre_tuned_rewards)
overall_mean_post = np.mean(post_tuned_rewards)

ax.hlines(overall_mean_pre, xmin=0.8, xmax=1.2, colors='blue', linestyles='dashed', linewidth=2, label='Mean Pre-Tuned')
ax.hlines(overall_mean_post, xmin=1.8, xmax=2.2, colors='green', linestyles='dashed', linewidth=2, label='Mean Post-Tuned')

# Highlight maximum values with annotation
max_pre = max(pre_tuned_rewards)
max_post = max(post_tuned_rewards)
ax.plot(1, max_pre, '^', markersize=10, label='Max Pre-Tuned', color='black')
ax.plot(2, max_post, '^', markersize=10, label='Max Post-Tuned', color='orange')

# Enhancing the plot
ax.set_title('Comparison of Total Rewards Before and After Tuning', fontsize=16)
ax.set_ylabel('Total Rewards', fontsize=14)
ax.set_xticklabels(['Pre-Tuned', 'Post-Tuned'], fontsize=12)
ax.set_yticklabels(np.round(ax.get_yticks(), 2), fontsize=12)

ax.yaxis.grid(True, linestyle='--', which='major', color='grey', alpha=0.7)
ax.set_axisbelow(True)  # Ensure grid is below plot elements

# Creating custom legend
legend_elements = [
    plt.Line2D([0], [0], color='lightgray', lw=4, label='Box (Quartiles)'),
    plt.Line2D([0], [0], color='red', lw=2, linestyle='-', label='Median'),
    plt.Line2D([0], [0], color='blue', lw=2, linestyle='dashed', label='Mean Pre-Tuned'),
    plt.Line2D([0], [0], color='green', lw=2, linestyle='dashed', label='Mean Post-Tuned'),
    plt.Line2D([0], [0], marker='^', color='w', markerfacecolor='black', markersize=10, label='Max Pre-Tuned'),
    plt.Line2D([0], [0], marker='^', color='w', markerfacecolor='orange', markersize=10, label='Max Post-Tuned')
]

ax.legend(handles=legend_elements, loc='upper left', fontsize=12,frameon=True, facecolor='white', edgecolor='black')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import scipy.stats as stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# Calculate mean and confidence interval for pre-tuned and post-tuned rewards
mean_pre, ci_lower_pre, ci_upper_pre = mean_confidence_interval(pre_tuned_rewards)
mean_post, ci_lower_post, ci_upper_post = mean_confidence_interval(post_tuned_rewards)

# Calculate means for plotting
mean_pre_series = [mean_pre] * len(pre_tuned_rewards)
mean_post_series = [mean_post] * len(post_tuned_rewards)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


mean_pre_series = np.mean(pre_tuned_rewards)
mean_post_series = np.mean(post_tuned_rewards)
ci_lower_pre, ci_upper_pre = np.percentile(pre_tuned_rewards, [2.5, 97.5])
ci_lower_post, ci_upper_post = np.percentile(post_tuned_rewards, [2.5, 97.5])


plt.style.use('seaborn-darkgrid')

plt.figure(figsize=(12, 7))

# Plotting mean lines
#plt.plot(mean_pre_series, label='Pre-Tuned Mean', color='black', linestyle='--', linewidth=2)
#plt.plot(mean_post_series, label='Post-Tuned Mean', color='darkorange', linestyle='--', linewidth=2)

# Shaded confidence interval
plt.fill_between(range(len(pre_tuned_rewards)), ci_lower_pre, ci_upper_pre, color='skyblue', alpha=0.4)
plt.fill_between(range(len(post_tuned_rewards)), ci_lower_post, ci_upper_post, color='navajowhite', alpha=0.4)

# Individual data points
plt.scatter(range(len(pre_tuned_rewards)), pre_tuned_rewards, color='blue', alpha=0.7, edgecolor='black', s=100, label='Pre-Tuned Data')
plt.scatter(range(len(post_tuned_rewards)), post_tuned_rewards, color='red', alpha=0.7, edgecolor='black', s=100, marker='x', label='Post-Tuned Data')

# Annotations for maximum and minimum points (as an example)
max_pre = max(pre_tuned_rewards)
min_post = min(post_tuned_rewards)
plt.annotate('Max Pre-Tuned', xy=(pre_tuned_rewards.index(max_pre), max_pre), xytext=(2, max_pre+30),
             arrowprops=dict(facecolor='blue', shrink=0.05), fontsize=12)
plt.annotate('Min Post-Tuned', xy=(post_tuned_rewards.index(min_post), min_post), xytext=(5, min_post-60),
             arrowprops=dict(facecolor='red', shrink=0.05), fontsize=12)

# Enhancing typography
plt.title('Total Rewards Over Runs Before and After Tuning with Confidence Intervals', fontsize=18)
plt.xlabel('Runs', fontsize=16)
plt.ylabel('Total Rewards', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Customizing the legend
plt.legend(loc='upper left', fontsize=14, frameon=True, facecolor='white', edgecolor='black')

plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()


In [ ]:
#plt.figure(figsize=(10, 6))
#plt.plot(pre_tuned_rewards, label='Pre-Tuned', marker='o')
#plt.plot(post_tuned_rewards, label='Post-Tuned', marker='x')
#plt.title('Total Rewards Over Runs Before and After Tuning')
#plt.xlabel('Runs')
#plt.ylabel('Total Rewards')
#plt.legend()
#plt.grid(True)
#plt.show()
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

plt.figure(figsize=(12, 7))
plt.plot(pre_tuned_rewards, label='Pre-Tuned', marker='o', markersize=8, linestyle='-', linewidth=2)
plt.plot(post_tuned_rewards, label='Post-Tuned', marker='x', markersize=8, linestyle='--', linewidth=2)

plt.title('Total Rewards Over Runs Before and After Tuning', fontsize=16)
plt.xlabel('Runs', fontsize=14)
plt.ylabel('Total Rewards', fontsize=14)

plt.text(x=max(len(pre_tuned_rewards), len(post_tuned_rewards)) - 2, y=min(min(pre_tuned_rewards), min(post_tuned_rewards)),
         s="Dashed lines indicate post-tuning performance", fontsize=12,
         bbox=dict(facecolor='white', alpha=0.5))

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc='upper left', bbox_to_anchor=(1, 1)) # Place legend outside
plt.grid(True, which='both', linestyle='--', linewidth=0.5) # Enhanced grid

# Show plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-darkgrid')

# Initialize figure
fig, ax = plt.subplots(figsize=(12, 8))

# Calculate moving averages for smooth trend lines
window_size = 3
pre_tuned_ma = np.convolve(pre_tuned_rewards, np.ones(window_size)/window_size, mode='valid')
post_tuned_ma = np.convolve(post_tuned_rewards, np.ones(window_size)/window_size, mode='valid')

# Plot moving averages with lines connecting data points
ax.plot(pre_tuned_rewards, 'o-', label='Pre-Tuned', color='blue', markersize=8, alpha=0.75, linewidth=2)
ax.plot(post_tuned_rewards, 'x--', label='Post-Tuned', color='orange', markersize=8, alpha=0.75, linewidth=2)

# Highlight the maximum reward points and annotate
max_pre = np.max(pre_tuned_rewards)
max_post = np.max(post_tuned_rewards)
max_pre_index = np.argmax(pre_tuned_rewards)
max_post_index = np.argmax(post_tuned_rewards)

ax.scatter(max_pre_index, max_pre, color='darkblue', s=100, zorder=5, edgecolor='black')
ax.scatter(max_post_index, max_post, color='darkorange', s=100, zorder=5, edgecolor='black')
ax.annotate(f'Max Pre: {max_pre}', (max_pre_index, max_pre), textcoords="offset points", xytext=(0,10), ha='center', fontsize=12)
ax.annotate(f'Max Post: {max_post}', (max_post_index, max_post), textcoords="offset points", xytext=(0,10), ha='center', fontsize=12)

# Enhancing the visual elements
ax.set_title('Total Rewards Over Runs Before and After Tuning', fontsize=18)
ax.set_xlabel('Runs', fontsize=16)
ax.set_ylabel('Total Rewards', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=14)

ax.legend(fontsize=14, loc='best',frameon=True, facecolor='white', edgecolor='black')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)

# To avoid clipping of the annotation
fig.tight_layout()

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-darkgrid')

plt.figure(figsize=(12, 8))

window_size = 3
pre_tuned_ma = np.convolve(pre_tuned_rewards, np.ones(window_size)/window_size, mode='valid')
post_tuned_ma = np.convolve(post_tuned_rewards, np.ones(window_size)/window_size, mode='valid')

plt.plot(pre_tuned_rewards, label='Pre-Tuned', marker='o', linestyle='', color='blue', alpha=0.5, markersize=8)
plt.plot(post_tuned_rewards, label='Post-Tuned', marker='x', linestyle='', color='orange', alpha=0.5, markersize=8)

plt.plot(range(window_size-1, len(pre_tuned_ma)+window_size-1), pre_tuned_ma, color='blue', linestyle='-', linewidth=2, alpha=0.7, label='Pre-Tuned MA')
plt.plot(range(window_size-1, len(post_tuned_ma)+window_size-1), post_tuned_ma, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Post-Tuned MA')

max_pre_index = np.argmax(pre_tuned_rewards)
max_post_index = np.argmax(post_tuned_rewards)
plt.scatter(max_pre_index, pre_tuned_rewards[max_pre_index], color='darkblue', s=100, zorder=5, edgecolor='black')
plt.scatter(max_post_index, post_tuned_rewards[max_post_index], color='darkorange', s=100, zorder=5, edgecolor='black')

plt.annotate(f'Max Pre: {pre_tuned_rewards[max_pre_index]}', (max_pre_index, pre_tuned_rewards[max_pre_index]), textcoords="offset points", xytext=(-10,10), ha='center')
plt.annotate(f'Max Post: {post_tuned_rewards[max_post_index]}', (max_post_index, post_tuned_rewards[max_post_index]), textcoords="offset points", xytext=(-10,10), ha='center')

plt.title('Total Rewards Over Runs Before and After Tuning', fontsize=18)
plt.xlabel('Runs', fontsize=16)
plt.ylabel('Total Rewards', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14, loc='best',frameon=True, facecolor='white', edgecolor='black')

plt.grid(True, which='major', linestyle='--', linewidth=0.75)
plt.grid(True, which='minor', linestyle=':', linewidth=0.5)
plt.minorticks_on()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-darkgrid')

plt.figure(figsize=(12, 8))

window_size = 3
pre_tuned_ma = np.convolve(pre_tuned_rewards, np.ones(window_size)/window_size, mode='valid')
post_tuned_ma = np.convolve(post_tuned_rewards, np.ones(window_size)/window_size, mode='valid')

plt.plot(pre_tuned_rewards, 'o-', label='Pre-Tuned', color='blue', markersize=6, linestyle='-', linewidth=1.5, alpha=0.7)
plt.plot(post_tuned_rewards, 'x--', label='Post-Tuned', color='orange', markersize=6, linestyle='--', linewidth=1.5, alpha=0.7)

plt.plot(range(window_size-1, len(pre_tuned_ma)+window_size-1), pre_tuned_ma, color='green', linestyle='-', linewidth=2, label='Pre-Tuned MA', alpha=0.7)
plt.plot(range(window_size-1, len(post_tuned_ma)+window_size-1), post_tuned_ma, color='red', linestyle='--', linewidth=2, label='Post-Tuned MA', alpha=0.7)


max_pre_index = np.argmax(pre_tuned_rewards)
max_post_index = np.argmax(post_tuned_rewards)
plt.scatter(max_pre_index, pre_tuned_rewards[max_pre_index], color='navy', s=100, zorder=5, edgecolor='black', alpha=0.8)
plt.scatter(max_post_index, post_tuned_rewards[max_post_index], color='chocolate', s=100, zorder=5, edgecolor='black', alpha=0.8)
plt.annotate(f'Max Pre: {pre_tuned_rewards[max_pre_index]}', (max_pre_index, pre_tuned_rewards[max_pre_index]), textcoords="offset points", xytext=(0,10), ha='center', fontsize=12)
plt.annotate(f'Max Post: {post_tuned_rewards[max_post_index]}', (max_post_index, post_tuned_rewards[max_post_index]), textcoords="offset points", xytext=(0,10), ha='center', fontsize=12)

plt.title('Total Rewards Over Runs Before and After Tuning', fontsize=18)
plt.xlabel('Runs', fontsize=16)
plt.ylabel('Total Rewards', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14, loc='best',frameon=True, facecolor='white', edgecolor='black')

plt.grid(True, which='major', linestyle='--', linewidth=0.75)
plt.grid(True, which='minor', linestyle=':', linewidth=0.5)
plt.minorticks_on()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(12, 8))

window_size = 3

# Function to calculate moving average and confidence interval
def moving_average_with_ci(data, window_size):
    ma = np.convolve(data, np.ones(window_size)/window_size, mode='valid')
    
    # Calculating standard deviation within the window for confidence intervals
    std_devs = [np.std(data[max(0, i-window_size+1):i+1]) for i in range(window_size-1, len(data))]
    ci = stats.norm.interval(0.95, loc=ma, scale=std_devs/np.sqrt(window_size))
    return ma, ci

# Applying the function to pre-tuned and post-tuned rewards
pre_tuned_ma, pre_tuned_ci = moving_average_with_ci(pre_tuned_rewards, window_size)
post_tuned_ma, post_tuned_ci = moving_average_with_ci(post_tuned_rewards, window_size)

# Plotting raw data
plt.plot(pre_tuned_rewards, 'o-', color='deepskyblue', markersize=6, alpha=0.5, label='Pre-Tuned Raw')
plt.plot(post_tuned_rewards, 'x--', color='coral', markersize=6, alpha=0.5, label='Post-Tuned Raw')

# Plotting moving averages
plt.plot(range(window_size-1, len(pre_tuned_ma)+window_size-1), pre_tuned_ma, color='navy', linestyle='-', linewidth=2, label='Pre-Tuned MA')
plt.plot(range(window_size-1, len(post_tuned_ma)+window_size-1), post_tuned_ma, color='darkred', linestyle='--', linewidth=2, label='Post-Tuned MA')

# Plotting confidence intervals
plt.fill_between(range(window_size-1, len(pre_tuned_ma)+window_size-1), pre_tuned_ci[0], pre_tuned_ci[1], color='deepskyblue', alpha=0.2, label='Pre-Tuned CI')
plt.fill_between(range(window_size-1, len(post_tuned_ma)+window_size-1), post_tuned_ci[0], post_tuned_ci[1], color='coral', alpha=0.2, label='Post-Tuned CI')

# Highlight and label the maximum points
max_pre_index = np.argmax(pre_tuned_rewards)
max_post_index = np.argmax(post_tuned_rewards)
plt.scatter([max_pre_index], [pre_tuned_rewards[max_pre_index]], color='blue', s=100, label='Max Pre-Tuned', zorder=5)
plt.scatter([max_post_index], [post_tuned_rewards[max_post_index]], color='red', s=100, label='Max Post-Tuned', zorder=5)
plt.text(max_pre_index, pre_tuned_rewards[max_pre_index], ' Max Pre', verticalalignment='bottom')
plt.text(max_post_index, post_tuned_rewards[max_post_index], ' Max Post', verticalalignment='bottom')

plt.title('Total Rewards Over Runs Before and After Tuning with Moving Averages and Confidence Intervals', fontsize=14)
plt.xlabel('Runs', fontsize=12)
plt.ylabel('Total Rewards', fontsize=12)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Move legend outside the plot
plt.tight_layout()
plt.show()


Pre-tuning Attacker

In [ ]:
def run_training_with_hyperparameters(gym_env, environment_properties,
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay):

    trained_attacker = learner.epsilon_greedy_search(
        gym_env,
        ep,
        a.QTabularLearner(ep, gamma=gamma, learning_rate=learning_rate, exploit_percentile=100),
        episode_count=episode_count,
        iteration_count=iteration_count,
        epsilon=epsilon,
        render=True,
        epsilon_multdecay=epsilon_multdecay,  # 0.999,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        verbosity=Verbosity.Quiet,
        title="Q-learning"
    )
    total_reward = sum([sum(rewards) for rewards in trained_attacker['all_episodes_rewards']])
    return total_reward


In [ ]:
import gym

# Create an instance of the CyberBattleChain environment without the defender
gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName"  
defender_agent = DefenderWrapper(gym_env,defender_name)

In [ ]:
random.seed(120394016)
# Parameter ranges
epsilon = 1
#gamma=np.linspace(0.05,0.09,10)
gamma= 0.0
#learning_rate=np.linspace(0.3,.35,10)
learning_rate=0.0
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=1000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0
episode_count=10
iteration_count=10
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.0
rewards=[]
for i in range(10):
    attackerreward=run_training_with_hyperparameters(gym_env, ep,
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay)
    rewards.append(attackerreward)

In [ ]:
rewards

In [ ]:
pre_tuned_rewards=[-224.0,
 -423.0,
 -228.0,
 -160.0,
 -156.0,
 -199.0,
 -287.0,
 -215.0,
 -331.0,
 -314.0]

Post-tuning Attacker

In [ ]:
random.seed(120394016)
# Parameter ranges
epsilon = 0.9629629629629622
#gamma=np.linspace(0.05,0.09,10)
gamma= 0.05888888888888889
#learning_rate=np.linspace(0.3,.35,10)
learning_rate=0.3222222222222222
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=5000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0513409942068965
episode_count=10
iteration_count=10
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.6631578947368421
rewards1=[]
for i in range(10):
    attackerreward=run_training_with_hyperparameters(gym_env, ep,
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay)
    rewards1.append(attackerreward)

In [ ]:
post_tuned_rewards=[-140.0,
 -261.0,
 -232.0,
 -158.0,
 -376.0,
 -356.0,
 -290.0,
 -165.0,
 -371.0,
 -410.0]

In [ ]:
random.seed(120394016)
# Parameter ranges
epsilon = 0.9629629629629622
#gamma=np.linspace(0.05,0.09,10)
gamma= 1-0.05888888888888889
#learning_rate=np.linspace(0.3,.35,10)
learning_rate=1-0.3222222222222222
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=5000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0513409942068965
episode_count=10
iteration_count=10
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.6631578947368421
rewards2=[]
for i in range(10):
    attackerreward=run_training_with_hyperparameters(gym_env, ep,
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay)
    rewards2.append(attackerreward)

In [ ]:
rewards2

In [ ]:
rewards6=[-1357.0,
 -1298.0,
 -1246.0,
 -1328.0,
 -1099.0,
 -1805.0,
 -1233.0,
 -1531.0,
 -1257.0,
 -1027.0]

In [ ]:
import numpy as np
from scipy import stats

# Step 1: Check for normality
print("Normality Check (Shapiro-Wilk Test):")
print("Pre-Tuned:", stats.shapiro(pre_tuned_rewards))
print("Post-Tuned:", stats.shapiro(post_tuned_rewards))

# Step 1: Check for equal variances
print("\nEqual Variances Check (Levene's Test):")
print(stats.levene(pre_tuned_rewards, post_tuned_rewards))

# Step 2: Choose and perform the test
# If both p-values from Shapiro and Levene's test are > 0.05
print("\nT-Test (if assumptions are met):")
print(stats.ttest_ind(pre_tuned_rewards, post_tuned_rewards))

# If any assumption is not met, use Mann-Whitney U test instead
print("\nMann-Whitney U Test (if assumptions are not met):")
print(stats.mannwhitneyu(pre_tuned_rewards, post_tuned_rewards))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 7))

# Enhanced color scheme for boxes
colors = ['#1f77b4', '#ff7f0e']

# Custom boxplots
box = plt.boxplot([pre_tuned_rewards, post_tuned_rewards], labels=['Pre-Tuned', 'Post-Tuned'],
            patch_artist=True,
            #notch=True,  # Adds a notch for median confidence interval
            meanline=True,
            showmeans=True,
            showfliers=True,  # Show outliers
            flierprops=dict(marker='o', markerfacecolor='red', markersize=6, linestyle='none'),
            medianprops=dict(linestyle='-', linewidth=2.5, color='yellow'),
            meanprops=dict(linestyle='--', linewidth=2, color='black'),
            whiskerprops=dict(color='gray', linewidth=2),
            capprops=dict(color='gray', linewidth=2))

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_edgecolor('black')  # Box edges
    patch.set_linewidth(2)  # Box edge linewidth

for i, data in enumerate([pre_tuned_rewards, post_tuned_rewards]):
    x = np.random.normal(i + 1, 0.04, size=len(data))
    plt.scatter(x, data, alpha=0.7, color='darkgreen', edgecolor='black', zorder=10)

plt.title('Comparison of Total Rewards Before and After Tuning', fontsize=18, fontweight='bold')
plt.ylabel('Total Rewards', fontsize=16, fontweight='regular')
plt.xticks(fontsize=14, fontweight='bold')
plt.yticks(fontsize=12, fontweight='regular')

# Grid and background
plt.gca().set_facecolor('#EAEAF2')  # Light background color
plt.grid(True, linestyle='-', linewidth=0.5, color='white', which='major')  # White grid for contrast
plt.minorticks_on()
plt.grid(True, linestyle=':', linewidth=0.3, color='white', which='minor')  # Minor grid

# Legend for custom elements
plt.legend([box["medians"][0], box["means"][0], box['fliers'][0]], ['Median', 'Mean'], loc='upper right', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))  # Enlarge the plot for better visibility

# Plotting with enhanced visibility and distinct markers
plt.plot(pre_tuned_rewards, label='Pre-Tuned', marker='o', markersize=8, linestyle='-', linewidth=2, color='royalblue')
plt.plot(post_tuned_rewards, label='Post-Tuned', marker='X', markersize=8, linestyle='--', linewidth=2, color='darkorange')

# Highlight the maximum values for each series
max_pre_index, max_pre_value = max(enumerate(pre_tuned_rewards), key=lambda x: x[1])
max_post_index, max_post_value = max(enumerate(post_tuned_rewards), key=lambda x: x[1])

plt.scatter(max_pre_index, max_pre_value, color='red', s=100, edgecolors='black', zorder=5, label='Max Pre-Tuned')
plt.scatter(max_post_index, max_post_value, color='green', s=100, edgecolors='black', zorder=5, label='Max Post-Tuned')

# Annotations for maximum values
plt.annotate(f'Max: {max_pre_value}', (max_pre_index, max_pre_value), textcoords="offset points", xytext=(0,10), ha='center', fontsize=12)
plt.annotate(f'Max: {max_post_value}', (max_post_index, max_post_value), textcoords="offset points", xytext=(0,10), ha='center', fontsize=12)

# Enhancing plot aesthetics
plt.title('Total Rewards Over Runs Before and After Tuning', fontsize=16)
plt.xlabel('Runs', fontsize=14)
plt.ylabel('Total Rewards', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)  # Adding a grid for better readability
plt.tight_layout() 
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 8))

mean_pre_tuned = np.mean(pre_tuned_rewards)
mean_post_tuned = np.mean(post_tuned_rewards)

std_pre_tuned = np.std(pre_tuned_rewards)
std_post_tuned = np.std(post_tuned_rewards)

plt.plot(pre_tuned_rewards, label='Pre-Tuned', marker='o', markersize=8, linestyle='-', linewidth=2, color='black')
plt.plot(post_tuned_rewards, label='Post-Tuned', marker='X', markersize=8, linestyle='--', linewidth=2, color='green')

plt.axhline(y=mean_pre_tuned, color='black', linestyle='-', alpha=0.7, label='Mean Pre-Tuned')
plt.axhline(y=mean_post_tuned, color='darkgreen', linestyle='--', alpha=0.7, label='Mean Post-Tuned')

max_pre_index, max_pre_value = max(enumerate(pre_tuned_rewards), key=lambda x: x[1])
max_post_index, max_post_value = max(enumerate(post_tuned_rewards), key=lambda x: x[1])
plt.annotate(f'Max Pre: {max_pre_value}', (max_pre_index, max_pre_value), textcoords="offset points", xytext=(0,10), ha='center')
plt.annotate(f'Max Post: {max_post_value}', (max_post_index, max_post_value), textcoords="offset points", xytext=(0,10), ha='center')

plt.title('Total Rewards Over Runs Before and After Tuning', fontsize=16)
plt.xlabel('Runs', fontsize=14)
plt.ylabel('Total Rewards', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.show()


# compare 10 episodes, 50 episodes, 10 iterations, 200 iterations

In [ ]:
from cyberbattle.agents.baseline.agent_wrapper import ActionTrackingStateAugmentation, StateAugmentation
from cyberbattle.agents.baseline.plotting import PlotTraining, plot_averaged_cummulative_rewards
import progressbar
import math
from cyberbattle.agents.baseline import learner
import sys
from cyberbattle.simulation.model import FirewallRule
import gym

# Create an instance of the CyberBattleChain environment without the defender
gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName" 
defender_agent = DefenderWrapper(gym_env,defender_name)
ep = w.EnvironmentBounds.of_identifiers(
    maximum_total_credentials=22,
    maximum_node_count=22,
    identifiers=gym_env.identifiers
)
#cyberbattlechain_defender.render_as_fig()

In [ ]:
# Parameter ranges
epsilon = 0.9629629629629622
#gamma=np.linspace(0.05,0.09,10)
gamma= 0.05888888888888889
#learning_rate=np.linspace(0.3,.35,10)
random.seed(120394016)
learning_rate=0.3222222222222222
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=5000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0513409942068965
episode_count=10
iteration_count=10
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.6631578947368421

trained_attacker = learner.epsilon_greedy_search(
        gym_env,
        ep,
        a.QTabularLearner(ep, gamma=0.05888888888888889, learning_rate=0.3222222222222222, exploit_percentile=100),
        episode_count=episode_count,
        iteration_count=iteration_count,
        epsilon=epsilon,
        render=True,
        epsilon_multdecay=epsilon_multdecay,  # 0.999,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        verbosity=Verbosity.Quiet,
        title="Q-learning"
    )

In [ ]:
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
random.seed(120394016)
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=10
iteration_count=10
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)
learner2 = DefenderQLearner(gym_env, defender_agent, ep, q_matrix, epsilon=0.9, gamma=0.015, learning_rate=.07)
random.seed(120394016)
#%matplotlib inline

trained_defender = epsilon_greedy_defender_training(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=learner2,
        episode_count=episode_count,  # Set the number of episodes for the training
        iteration_count=iteration_count,  # Set the number of iterations per episode
        epsilon=epsilon,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        epsilon_multdecay=epsilon_multdecay,
        verbosity=logging.INFO,
        render=True,
        plot_episodes_length=True,
        title="Defender Q-Learning Training"
    )

In [ ]:
import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName" 
defender_agent = DefenderWrapper(gym_env,defender_name)

gym_environment = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    ),
    defender_agent=defender_agent
)

#this was made due to issues where the attacker q learner needed to use a wrapper environment
from cyberbattle.agents.baseline.agent_wrapper import AgentWrapper, StateAugmentation
def wrap_environment(env):
    initial_observation = env.reset()  
    state = StateAugmentation(initial_observation)  
    return AgentWrapper(env, state)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class SimulationTracker:
    def __init__(self):
        self.attacker_rewards = []
        self.defender_rewards = []
        self.actions_taken = {'attacker': [], 'defender': []}
        self.exploration_actions = {'attacker': 0, 'defender': 0}
        self.exploitation_actions = {'attacker': 0, 'defender': 0}
        self.episode_lengths = []
        self.reward_variability = []
        self.state_visitation_frequencies = {}
        self.temporal_differences = []
        self.infected_nodes = []

    def log_episode(self, attacker_reward, defender_reward, episode_length, episode_rewards):
        self.attacker_rewards.append(attacker_reward)
        self.defender_rewards.append(defender_reward)
        self.episode_lengths.append(episode_length)
        self.reward_variability.append(np.var(episode_rewards))
        #self.temporal_differences.append(temporal_difference)

    def log_action(self, role, action, exploration):
        action_str = str(action)  # Convert the action dictionary to a string
        self.actions_taken[role].append(action_str)
        if exploration:
            self.exploration_actions[role] += 1
        else:
            self.exploitation_actions[role] += 1


    def log_state_visitation(self, state):
        if state not in self.state_visitation_frequencies:
            self.state_visitation_frequencies[state] = 1
        else:
            self.state_visitation_frequencies[state] += 1

    def log_infected_nodes(self, nodes_count):
        self.infected_nodes.append(nodes_count)

    def print_summary(self):
        print("Simulation Summary:")
        print(f"Average Attacker Reward: {np.mean(self.attacker_rewards)}")
        print(f"Average Defender Reward: {np.mean(self.defender_rewards)}")
        print(f"Average Episode Length: {np.mean(self.episode_lengths)} steps")
        print(f"Average Reward Variability: {np.mean(self.reward_variability)}")
        print(f"Average Temporal Difference: {np.mean(self.temporal_differences)}")
        print(f"Average Infected Nodes per Episode: {np.mean(self.infected_nodes)}")

        print("\nExploration vs. Exploitation:")
        for role in ['attacker', 'defender']:
            total_actions = self.exploration_actions[role] + self.exploitation_actions[role]
            if total_actions > 0:
                print(f"{role.capitalize()} Exploration: {self.exploration_actions[role] / total_actions:.2f}")
                print(f"{role.capitalize()} Exploitation: {self.exploitation_actions[role] / total_actions:.2f}")

        self._print_action_distributions()

        self._plot_rewards()

    def _print_action_distributions(self):
        print("\nAction Distributions:")
        for role, actions in self.actions_taken.items():
            unique_actions, counts = np.unique(actions, return_counts=True)
            print(f"{role.capitalize()} Actions:")
            for action, count in zip(unique_actions, counts):
                print(f"Action {action}: {count} times")


    def _plot_rewards(self):
        plt.figure(figsize=(10, 5))
        plt.plot(self.attacker_rewards, label='Attacker Rewards')
        plt.plot(self.defender_rewards, label='Defender Rewards')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.title('Rewards per Episode for Attacker and Defender')
        plt.legend()
        plt.show()


In [ ]:
attacker_learner = trained_attacker['learner']
defender_learner = trained_defender['learner']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
random.seed(120394016)
# Simulation Parameters
number_of_episodes = 10
max_steps_per_episode = 10
attacker_epsilon = 0.1
defender_epsilon = 0.1

# Reward Tracking
attacker_rewards_per_episode = []
defender_rewards_per_episode = []
win_rates = {'attacker': [], 'defender': []}
actions_taken_count = {'attacker': [], 'defender': []}
exploration_vs_exploitation = {'exploration': [], 'exploitation': []}
episode_lengths = []
state_visitation_frequencies = {}
reward_variability = []
defender_successful_actions = []
defender_failed_actions = []
defender_actions_taken = []
tracker = SimulationTracker() #instantiate simulation tracker
wrapped_env = wrap_environment(gym_environment)
win_counts = {'attacker': 0, 'defender': 0}
#main simulation loop
random.seed(120394016)
for episode in range(number_of_episodes):
    observation = wrapped_env.reset()
    attacker_total_reward = 0
    defender_total_reward = 0
    steps_this_episode=0
    episode_rewards = []  #track rewards for this episode
    for step in range(max_steps_per_episode):
        steps_this_episode +=1
        #attacker's Turn
        exploration = np.random.random() < attacker_epsilon
        if exploration:
            _, attacker_action, _ = attacker_learner.explore(wrapped_env)
        else:
            _, attacker_action, _ = attacker_learner.exploit(wrapped_env, observation)
        #print(attacker_action)
        if attacker_action is not None:
            observation, attacker_reward, done, _ = gym_environment.step(attacker_action)
            attacker_total_reward += attacker_reward
            print(_)
            actions_taken_count['attacker'].append(attacker_action)  # Track action
            episode_rewards.append(attacker_reward)  # Track reward variability
            tracker.log_action('attacker', attacker_action, exploration)
            #actions_taken_count['attacker'][attacker_action] = actions_taken_count['attacker'].get(attacker_action, 0) + 1


        # Defender's Turn
        exploration = np.random.random() < defender_epsilon
        if exploration:
            _, defender_action, _ = defender_learner.explore(wrapped_env)
        else:
            _, defender_action, _ = defender_learner.exploit(wrapped_env, observation)

        if defender_action is not None:
            observation, defender_reward, done, _ = gym_environment.step(defender_action)
            #action= _['defender_info']['action']
            defender_total_reward += defender_reward
            actions_taken_count['defender'].append(defender_action)  # Track action
            episode_rewards.append(defender_reward)  # Track reward variability
            tracker.log_action('defender', defender_action, exploration)
            #actions_taken_count['defender'][defender_action] = actions_taken_count['defender'].get(defender_action, 0) + 1
        #logging
        # After each episode
        tracker.log_episode(attacker_total_reward, defender_total_reward, steps_this_episode, episode_rewards)
        tracker.print_summary()
        tracker._plot_rewards()

        if done:
            break

    defender_info = _['defender_info']
    if defender_info:
        defender_actions_taken.append(defender_info['performance_metrics']['actions_taken'])
        defender_successful_actions.append(defender_info['performance_metrics']['successful_actions'])
        defender_failed_actions.append(defender_info['performance_metrics']['failed_actions'])

    attacker_rewards_per_episode.append(attacker_total_reward)
    defender_rewards_per_episode.append(defender_total_reward)
    episode_lengths.append(steps_this_episode)
    reward_variability.append(np.var(episode_rewards))
    print(f"Episode {episode + 1}: Attacker Reward = {attacker_total_reward}, Defender Reward = {defender_total_reward}")

wrapped_env.close()

In [ ]:
attacker10_rewards_per_episode = attacker_rewards_per_episode
defender10_rewards_per_episode = defender_rewards_per_episode
win_rates10=win_rates
defender_successful_actions10 = defender_successful_actions
defender_failed_actions10 = defender_failed_actions
defender_actions_taken10 = defender_actions_taken

epsiode 50x200

In [ ]:
#this was made due to issues where the attacker q learner needed to use a wrapper environment
from cyberbattle.agents.baseline.agent_wrapper import AgentWrapper, StateAugmentation
def wrap_environment(env):
    initial_observation = env.reset()
    state = StateAugmentation(initial_observation)
    return AgentWrapper(env, state)

import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName"
defender_agent = DefenderWrapper(gym_env,defender_name)

gym_environment = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    ),
    defender_agent=defender_agent
)



In [ ]:
#parameter ranges
epsilon = 0.9629629629629622
#gamma=np.linspace(0.05,0.09,10)
gamma= 0.05888888888888889
#learning_rate=np.linspace(0.3,.35,10)
learning_rate=0.3222222222222222
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=5000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0513409942068965
episode_count=50
iteration_count=200
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.6631578947368421

trained_attacker = learner.epsilon_greedy_search(
        gym_env,
        ep,
        a.QTabularLearner(ep, gamma=0.05888888888888889, learning_rate=0.3222222222222222, exploit_percentile=100),
        episode_count=episode_count,
        iteration_count=iteration_count,
        epsilon=epsilon,
        render=True,
        epsilon_multdecay=epsilon_multdecay,  # 0.999,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        verbosity=Verbosity.Quiet,
        title="Q-learning"
    )

In [ ]:
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=50
iteration_count=200
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)
learner2 = DefenderQLearner(gym_env, defender_agent, ep, q_matrix, epsilon=0.9, gamma=0.015, learning_rate=.07)
random.seed(120394016)
#%matplotlib inline

trained_defender = epsilon_greedy_defender_training(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=learner2,
        episode_count=episode_count, 
        iteration_count=iteration_count, 
        epsilon=epsilon,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        epsilon_multdecay=epsilon_multdecay,
        verbosity=logging.INFO,
        render=True,
        plot_episodes_length=True,
        title="Defender Q-Learning Training"
    )

In [ ]:
#this was made due to issues where the attacker q learner needed to use a wrapper environment
from cyberbattle.agents.baseline.agent_wrapper import AgentWrapper, StateAugmentation
def wrap_environment(env):
    initial_observation = env.reset()
    state = StateAugmentation(initial_observation)
    return AgentWrapper(env, state)

import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName"
defender_agent = DefenderWrapper(gym_env,defender_name)

gym_environment = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    ),
    defender_agent=defender_agent
)



In [ ]:
attacker_learner = trained_attacker['learner']
defender_learner = trained_defender['learner']

10x10

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#simulation parameters
number_of_episodes = 50
max_steps_per_episode = 200
attacker_epsilon = 0.1
defender_epsilon = 0.1

#reward tracking
attacker_rewards_per_episode = []
defender_rewards_per_episode = []
win_rates = {'attacker': [], 'defender': []}
actions_taken_count = {'attacker': [], 'defender': []}
exploration_vs_exploitation = {'exploration': [], 'exploitation': []}
episode_lengths = []
state_visitation_frequencies = {}
reward_variability = []
defender_successful_actions = []
defender_failed_actions = []
defender_actions_taken = []
tracker = SimulationTracker() #instantiate simulation tracker
wrapped_env = wrap_environment(gym_environment)
win_counts = {'attacker': 0, 'defender': 0}
# Main Simulation Loop
random.seed(120394016)
for episode in range(number_of_episodes):
    observation = wrapped_env.reset()
    attacker_total_reward = 0
    defender_total_reward = 0
    steps_this_episode=0
    episode_rewards = []  # Track rewards for this episode
    for step in range(max_steps_per_episode):
        steps_this_episode +=1
        # Attacker's Turn
        exploration = np.random.random() < attacker_epsilon
        if exploration:
            _, attacker_action, _ = attacker_learner.explore(wrapped_env)
        else:
            _, attacker_action, _ = attacker_learner.exploit(wrapped_env, observation)
        #print(attacker_action)
        if attacker_action is not None:
            observation, attacker_reward, done, _ = gym_environment.step(attacker_action)
            attacker_total_reward += attacker_reward
            print(_)
            actions_taken_count['attacker'].append(attacker_action)  # Track action
            episode_rewards.append(attacker_reward)  # Track reward variability
            tracker.log_action('attacker', attacker_action, exploration)
            #actions_taken_count['attacker'][attacker_action] = actions_taken_count['attacker'].get(attacker_action, 0) + 1


        # Defender's Turn
        exploration = np.random.random() < defender_epsilon
        if exploration:
            _, defender_action, _ = defender_learner.explore(wrapped_env)
        else:
            _, defender_action, _ = defender_learner.exploit(wrapped_env, observation)

        if defender_action is not None:
            observation, defender_reward, done, _ = gym_environment.step(defender_action)
            #action= _['defender_info']['action']
            defender_total_reward += defender_reward
            actions_taken_count['defender'].append(defender_action)  # Track action
            episode_rewards.append(defender_reward)  # Track reward variability
            tracker.log_action('defender', defender_action, exploration)
            #actions_taken_count['defender'][defender_action] = actions_taken_count['defender'].get(defender_action, 0) + 1
        #logging
        # After each episode
        tracker.log_episode(attacker_total_reward, defender_total_reward, steps_this_episode, episode_rewards)
        tracker.print_summary()
        tracker._plot_rewards()

        if done:
            break

    defender_info = _['defender_info']
    if defender_info:
        defender_actions_taken.append(defender_info['performance_metrics']['actions_taken'])
        defender_successful_actions.append(defender_info['performance_metrics']['successful_actions'])
        defender_failed_actions.append(defender_info['performance_metrics']['failed_actions'])

    attacker_rewards_per_episode.append(attacker_total_reward)
    defender_rewards_per_episode.append(defender_total_reward)
    episode_lengths.append(steps_this_episode)
    reward_variability.append(np.var(episode_rewards))
    print(f"Episode {episode + 1}: Attacker Reward = {attacker_total_reward}, Defender Reward = {defender_total_reward}")

wrapped_env.close()

In [ ]:
attacker50_rewards_per_episode = attacker_rewards_per_episode
defender50_rewards_per_episode = defender_rewards_per_episode
win_rates50=win_rates
defender_successful_actions50 = defender_successful_actions
defender_failed_actions50 = defender_failed_actions
defender_actions_taken50 = defender_actions_taken

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.hist(attacker10_rewards_per_episode, alpha=0.5, label='10 Episodes')
plt.hist(attacker50_rewards_per_episode, alpha=0.5, label='50 Episodes')
plt.legend()
plt.title('Attacker Rewards Distribution')

plt.subplot(1, 2, 2)
plt.hist(defender10_rewards_per_episode, alpha=0.5, label='10 Episodes')
plt.hist(defender50_rewards_per_episode, alpha=0.5, label='50 Episodes')
plt.legend()
plt.title('Defender Rewards Distribution')

plt.tight_layout()
plt.show()


In [ ]:
#attacker rewards comparison
t_stat, p_value = stats.ttest_ind(attacker10_rewards_per_episode, attacker50_rewards_per_episode, equal_var=False)
print(f'Attacker Rewards: t-statistic={t_stat:.2f}, p-value={p_value:.4f}')

#defender rewards comparison
t_stat, p_value = stats.ttest_ind(defender10_rewards_per_episode, defender50_rewards_per_episode, equal_var=False)
print(f'Defender Rewards: t-statistic={t_stat:.2f}, p-value={p_value:.4f}')


In [ ]:
#calculate percentage of successful actions for each simulation set
def percentage_successful(actions_taken, successful_actions):
    return [successful / total if total != 0 else 0 for successful, total in zip(successful_actions, actions_taken)]

percentage_successful_10 = percentage_successful(defender_actions_taken10, defender_successful_actions10)
percentage_successful_50 = percentage_successful(defender_actions_taken50, defender_successful_actions50)

# Plotting
plt.figure(figsize=(12, 6))
episodes_10 = np.arange(1, len(percentage_successful_10) + 1)
episodes_50 = np.arange(1, len(percentage_successful_50) + 1)
plt.plot(episodes_10, percentage_successful_10, label='10 Episodes @ 10 Iterations', marker='o')
plt.plot(episodes_50, percentage_successful_50, label='50 Episodes @ 200 Iterations', marker='x')

plt.xlabel('Episode')
plt.ylabel('Percentage of Successful Actions')
plt.title('Percentage of Successful Defender Actions')
plt.legend(frameon=True,facecolor='white')
plt.grid(True)
plt.show()


In [ ]:
#mann-whitney U Test
u_stat, p_value = stats.mannwhitneyu(percentage_successful_10, percentage_successful_50, alternative='less')

print(f'Mann-Whitney U Test: U-Statistic={u_stat:.2f}, p-value={p_value:.4f}')
